**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Para resolver los problemas presentados, use el siguiente conjunto de datos. Use [gradetool](gradetool.md) para verificar las respuestas del notebook.

In [1]:
%%writefile data.tsv
E	b,g,f	jjj#3,bbb#0,ddd#9,ggg#8,hhh#2
A	a,f,c	ccc#2,ddd#0,aaa#3,hhh#9
B	f,e,a,c	ddd#2,ggg#5,ccc#6,jjj#1
A	a,b	hhh#9,iii#5,eee#7,bbb#1
C	f,g,d,a	iii#6,ddd#5,eee#4,jjj#3
A	c,d	bbb#2,hhh#0,ccc#4,fff#1,aaa#7
A	g,d,a	aaa#5,fff#8,ddd#2,iii#0,jjj#7,ccc#1
B	b,a	fff#3,hhh#1,ddd#2
E	d,e,a,f	eee#4,ccc#5,iii#9,fff#7,ggg#6,bbb#0
B	d,b,g,f	bbb#7,jjj#9,fff#5,iii#4,ggg#2,eee#3
C	d,c,f,b	hhh#6,eee#4,iii#0,fff#2,jjj#1
C	d,e,a,c	bbb#7,iii#6,ggg#9
D	g,e,f,b	bbb#9,aaa#3,ccc#6,fff#4,eee#2
E	c,f	aaa#8,ddd#5,jjj#1
B	d,b	ccc#0,jjj#6,fff#7,ddd#3,aaa#2
D	f,e	ccc#0,eee#6,bbb#9,ddd#3
E	e,b,f	bbb#6,iii#3,hhh#5,fff#4,ggg#9,ddd#2
D	g,a	hhh#4,jjj#5,ccc#9
E	e,c,f,a	ccc#1,iii#6,fff#9
E	e,a	bbb#9,aaa#3,fff#1
E	e,f	ddd#9,iii#2,aaa#4
E	c,b,g	ccc#5,fff#8,iii#7
D	c,f,a	eee#3,jjj#2,ddd#7
A	f,a,d	jjj#1,ggg#0,ccc#7,ddd#9,bbb#3
E	c,d	jjj#6,ccc#0,aaa#1,hhh#9,iii#7,ggg#8
E	e,d,c	fff#3,eee#6,iii#4,bbb#7,ddd#0,ccc#1
A	a,e,f	fff#0,ddd#5,ccc#4
E	c,a,g	ggg#6,hhh#3,ddd#9,ccc#0,jjj#7
A	f,e	hhh#6,jjj#0,eee#5,iii#7,ccc#3
C	f,c,a,g	eee#1,fff#4,aaa#2,ccc#7,ggg#0,ddd#6
A	b,f	ccc#6,aaa#9,eee#5,ddd#0,bbb#3
D	b,f	bbb#7,hhh#1,aaa#6,iii#4,fff#9,ddd#5
E	a,c	fff#3,ccc#1,ggg#2,eee#5
B	b,f,c	iii#7,ggg#3,ddd#0,jjj#8,hhh#5,ccc#1
B	f,a,e	hhh#6,ccc#3,jjj#0,bbb#8,ddd#7
D	a,f	aaa#0,fff#5,ddd#3
B	c,a	ddd#5,jjj#2,iii#7,ccc#0,bbb#4
C	c,a,e,f	eee#0,fff#2,hhh#6
E	e,d	fff#9,iii#2,eee#0
E	f,a,d	hhh#8,ggg#3,jjj#5

Overwriting data.tsv


In [2]:
%load_ext bigdata
%timeout 500
%hive_init

500
Hive initialized!


In [3]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY '\t'
COLLECTION ITEMS TERMINATED BY ','
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

-- SELECT * FROM t0;

# Problema 1

Escriba una consulta que calcule la cantidad de registros por clave de la columna 3. En otras palabras, cuántos registros hay que tengan la clave `aaa`?

In [4]:
%%hive
SELECT k, COUNT(k)
FROM
(SELECT EXPLODE(c3) AS (k,v)
FROM t0) a
GROUP BY k;

aaa	13
bbb	16
ccc	23
ddd	23
eee	15
fff	20
ggg	13
hhh	16
iii	18
jjj	18


# Problema 2

Escriba una consulta que retorne la primera columna, la cantidad de elementos en la columna 2 y la cantidad de elementos en la columna 3

In [5]:
%%hive
SELECT c1, SIZE(c2), SIZE(c3)
FROM t0;

E	3	5
A	3	4
B	4	4
A	2	4
C	4	4
A	2	5
A	3	6
B	2	3
E	4	6
B	4	6
C	4	5
C	4	3
D	4	5
E	2	3
B	2	5
D	2	4
E	3	6
D	2	3
E	4	3
E	2	3
E	2	3
E	3	3
D	3	3
A	3	5
E	2	6
E	3	6
A	3	3
E	3	5
A	2	5
C	4	6
A	2	5
D	2	6
E	2	4
B	3	6
B	3	5
D	2	3
B	2	5
C	4	3
E	2	3
E	3	3


# Problema 3

Escriba una consulta que compute la cantidad de registros por letra de la columna 2 y clave de la columna 3; esto es, por ejemplo, la cantidad de registros en tienen la letra `a` en la columna 2 y la clave `aaa` en la columna 3 es:

     a    aaa    5

In [46]:
%%hive
SELECT s, k, COUNT(c1)
FROM t0
LATERAL VIEW explode(c2) t1 AS s
LATERAL VIEW explode(c3) t2 AS k,v
GROUP BY s, k;

a	aaa	5
a	bbb	7
a	ccc	13
a	ddd	13
a	eee	7
a	fff	10
a	ggg	8
a	hhh	8
a	iii	7
a	jjj	10
b	aaa	4
b	bbb	7
b	ccc	5
b	ddd	7
b	eee	5
b	fff	8
b	ggg	4
b	hhh	7
b	iii	7
b	jjj	5
c	aaa	5
c	bbb	4
c	ccc	12
c	ddd	9
c	eee	6
c	fff	8
c	ggg	7
c	hhh	7
c	iii	8
c	jjj	8
d	aaa	4
d	bbb	6
d	ccc	7
d	ddd	5
d	eee	6
d	fff	8
d	ggg	6
d	hhh	4
d	iii	9
d	jjj	8
e	aaa	3
e	bbb	8
e	ccc	9
e	ddd	7
e	eee	7
e	fff	9
e	ggg	4
e	hhh	4
e	iii	8
e	jjj	3
f	aaa	8
f	bbb	10
f	ccc	13
f	ddd	17
f	eee	11
f	fff	11
f	ggg	9
f	hhh	10
f	iii	10
f	jjj	12
g	aaa	3
g	bbb	3
g	ccc	6
g	ddd	5
g	eee	4
g	fff	5
g	ggg	4
g	hhh	3
g	iii	4
g	jjj	6


---

In [ ]:
!rm data.* pig_*